# 발표 소단원 리스트

9-14,9-18,9-22,10-1,10-5,10-9,10-13

# 전체 요약자료

-


# 문항별 자료 
- (9-14) 클래스의 attribute의 정의 순서 기억하기 : metaclass 이용하기

- (9-18) type.new_class로 class 정의하기

- (9-22) Context manager 정의하기

- (10-1) 위계질서형의 패키지 구현하기 

- (10-5) Namespace package 정의와 장단점 및 만드는 방법

- (10-9) sys.path에 디렉토리 추가하기

- (10-13) 개별 계정에 패키지 설치하기 : --user 옵션 쓰기

### (9-14) Capturing Class Attribute Definition Order
- (sol) metaclass 정의하기

In [1]:
from collections import OrderedDict

# A set of descriptors for various types
class Typed:
    _expected_type = type(None)
    def __init__(self,name = None):
        self._name = name
        
    def __set__(self,instance,value):
        if not isinstance(value,self._expected_type):
            raise TypeError('Expected '+str(self._expected_type))
        instance.__dict__[self._name] = value
        
class Integer(Typed):
    _expected_type = int
    
class Float(Typed):
    _expected_type = float
    
class String(Typed):
    _expected_type = str
    
class OrderedMeta(type):
    def __new__(cls,clsname,bases,clsdict):
        d = dict(clsdict)
        order = []
        for name,value in clsdict.items():
            if isinstance(value,Typed):
                value._name = name
                order.append(name)
        
        d['_order'] = order
        return type.__new__(cls,clsname,bases,d)
    
    @classmethod
    def __prepare__(cls,clsname,bases):
        return OrderedDict() # bases class의 정의를 알아서 담나??
    
class Structure(metaclass = OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self,name)) for name in self._order)
    
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()
    
    def __init__(self,name,shares,price):
        self.name = name
        self.shares = shares
        self.price = price
        
s = Stock('GOOG',100,490.1)
s.name

'GOOG'

In [2]:
s.as_csv()

'GOOG,100,490.1'

In [3]:
t = Stock('AAPL','a lot',610.23)

TypeError: Expected <class 'int'>

In [4]:
# 2번째 예제 : 중복 정의 방지하기
from collections import OrderedDict

class NoDupOrderedDict(OrderedDict):
    def __init__(self,clsname):
        self.clsname = clsname
        super().__init__()
        
    def __setitem__(self,name,value):
        if name in self:
            raise TypeError('{} already defined in {}'.format(name,self.clsname))
        super().__setitem__(name,value)
        
class OrderedMeta(type):
    def __new__(cls,clsname,bases,clsdict):
        d = dict(clsdict)
        d['_order'] = [name for name in clsdict if name[0] != '_']
        return type.__new__(cls,clsname,bases,d)
    
    @classmethod
    def __prepare__(cls,clsname,bases):
        return NoDupOrderedDict(clsname)
    
class A(metaclass = OrderedMeta):
    def spam(self):
        pass
    def spam(self):
        pass

TypeError: spam already defined in A

### (9-18) Defining Classes Programmatically
- (sol) types.new_class()

In [5]:
def __init__(self,name,shares,price):
    self.name = name
    self.shares = shares
    self.price = price
    
def cost(self):
    return self.shares * self.price

cls_dict = {
    '__init__' : __init__,
    'cost' : cost,
}

In [12]:
## class 정의하는 또다른 방식
import types
Stock = types.new_class('Stock',(),{},lambda ns:ns.update(cls_dict))
Stock.__module__ = __name__

In [13]:
s = Stock('ACME',50,91.1)
s

In [11]:
s.cost()

4555.0

In [14]:
import abc
Stock = types.new_class('Stock',(),{'metaclass':abc.ABCMeta},lambda ns : ns.update(cls_dict))

In [15]:
Stock.__module__ = __name__
Stock

__main__.Stock

In [16]:
type(Stock)

abc.ABCMeta

In [ ]:
class Spam(Base,debug=True,typecheck=False):
    pass

# ==

Spam = types.new_class('Spam',(Base,),{'debug':True,'typecheck':False},
                      lambda ns : ns.update(cls_dict))

### (9-22) Defining Context Managers the Easy Way
- (sol) with statement!!

In [17]:
import time
from contextlib import contextmanager

@contextmanager
def timethis(label):
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
        print('{} : {}'.format(label,end-start))
        
# example use
with timethis('counting'):
    n = 10000000
    while n > 0:
        n -= 1

counting : 0.9723963737487793


In [18]:
## works only for no error occurs
@contextmanager
def list_transaction(orig_list):
    working = list(orig_list)
    yield working
    orig_list[:] = working

In [19]:
items = [1,2,3]
with list_transaction(items) as working:
    working.append(4)
    working.append(5)
    
items

[1, 2, 3, 4, 5]

In [21]:
with list_transaction(items) as working:
    working.append(6)
    working.append(7)
    raise RuntimeError('oops')

RuntimeError: oops

In [22]:
items

[1, 2, 3, 4, 5]

In [ ]:
## enter,exit method 정의 가능
import time
class timethis:
    def __init__(self,label):
        self.label = label
        
    def __enter__(self):
        self.start = time.time()
        
    def __exit__(self,exc_ty,exc_val,exc_tb):
        end = time.time()
        print('{}: {}'.format(self.label,end-self.start))

### (10-1) Making a Hierarchical Package of Modules
- (sol) 폴더 내 폴더 만들기
- __init__.py 파일 만들어주기

graphics/
    __init__.py
    primitive/
        __init__.py
        line.py
        fill.py
        text.py
    formats/
        __init__.py
        png.py
        jpg.py
        

In [ ]:
import graphics.primitive.text
from graphics.primitive import text
## 이 사이에 graphics, primitive 폴더 내의 모든 init을 실행시킴
## primitive 폴더 내의 init에 from . import jpg 걸면 -> import graphics.primive만 실행시켜도 효과 발생!!
## __init__.py 없으면 그냥 "namespace package"가 생성됨 (10.5에서 자세히 다룸)

### (10-5) Making Separate Directories of Code Import Under a Common Namespace
- (sol) sys.path.extend(여러 디렉토리)
- 파일분할, 여러 서브 작업들의 통합, add-ons
- __init__.py가 탑레벌 디렉토리에 정의되어 있지 않아야함!

foo-package/
    spam/
        blah.py

bar-package/
    spam/
        grok.py

In [1]:
import sys
sys.path.extend(['foo-package','bar-package'])

In [2]:
import spam.blah
import spam.grok
##라고 쓰고 싶을 때

In [3]:
import spam
spam.__path__ #하면 경로가 나오고,추가도 가능

_NamespacePath(['foo-package\\spam', 'bar-package\\spam'])

In [4]:
spam #쳐보면 namespace라고 출력됨

<module 'spam' (namespace)>

### (10-9) Adding Directories to sys.path
- 다른 경로에 있는 뭔가를 import 하고 싶을때, sys.path에 추가하기

In [ ]:
## sys.path에 경로 추가하는 2가지 방법
## 1. PYTHONPATH에 값 대입
# bash % env PYTHONPATH = /some/dir:/other/dir ptyhon3
import sys
sys.path #찍어보면 저 2개 들어가 있음

## 2. site-packages 폴더에 .pth파일 넣기
## myapplication.pth
# /some/dir
# /other/dir 
## 이런식으로 쓰면 됨

In [ ]:
import sys
sys.path.insert(0,'/some/dir')
sys.path.insert(0,'/other/dir') #이런 방식은 비추

### (10-13) Installing Packages just for yourself
- 로그인 옵션에서 자기 계정만을 의미하는 듯
- python은 ~/.local/lib/python 버전/site-packages 경로에 per-user 설치경로가 담겨 있음
- (sol) pip install --uesr 패키지 : 하면 site-packages보다 per-user 경로에 먼저 저장됨
- 자세한건 pip과 같은 third-party package manager의 작동방식을 알아봐야함